In [4]:
import pandas as pd
import numpy as np

from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmspe(y_true, y_pred):
  """
  Calculate Root Mean Squared Percentage Error (RMSPE)
  """
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  
  # Neglect 0 y_true values
  mask = y_true != 0
  y_true = y_true[mask]
  y_pred = y_pred[mask]

  if len(y_true) == 0:
    return 0  # Return 0 if there are no non-zero y_true values

  # Calculate the percentage errors
  percentage_errors = (y_true - y_pred) / y_true
  
  # Calculate the squared percentage errors
  squared_percentage_errors = percentage_errors ** 2
  
  # Calculate the mean squared percentage error
  mean_squared_percentage_error = np.mean(squared_percentage_errors)
  
  # Calculate the root mean squared percentage error
  root_mean_squared_percentage_error = np.sqrt(mean_squared_percentage_error)
  
  return root_mean_squared_percentage_error

/Users/skalkan/Desktop/rossmann/rossmann_case_study/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [6]:
df = pd.read_csv('../data/raw/train.csv', index_col = 'Date')
df = df.drop(columns = ['Id'])
df = df.sort_index()

/var/folders/wy/rd6_hk5s6xx5sy02hxjsz2gr0000gn/T/ipykernel_29759/2532742017.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/train.csv', index_col = 'Date')


In [7]:
# Group the data by 'Store' and count the number of non-null values in each column
store_counts = df.groupby('Store').count()

# Find the store with the most non-null values in all columns
most_available_store = store_counts.sum(axis=1).idxmax()

# Select the data for the most available store
df_selected = df[df['Store'] == most_available_store]

In [8]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 928 entries, 2013-01-01 to 2015-07-17
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Store          928 non-null    int64 
 1   DayOfWeek      928 non-null    int64 
 2   Sales          928 non-null    int64 
 3   Customers      928 non-null    int64 
 4   Open           928 non-null    int64 
 5   Promo          928 non-null    int64 
 6   StateHoliday   928 non-null    object
 7   SchoolHoliday  928 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 65.2+ KB


In [9]:
# Prepare the data for Prophet
df_prophet = df_selected[['Sales']].reset_index()
df_prophet.columns = ['ds', 'y']

# Split the data into train and test sets
train_size = int(len(df_prophet) * 0.8)
train_data = df_prophet[:train_size]
test_data = df_prophet[train_size:]

# Create and fit the Prophet model
model = Prophet()
model.fit(train_data)

# Make predictions on the test data
predictions = model.predict(test_data[['ds']])

# Evaluate the model
rmse = sqrt(mean_squared_error(test_data['y'], predictions['yhat']))
print('RMSE:', rmse)

rmspe_score = rmspe(test_data['y'], predictions['yhat'])
print('RMSPE:', rmspe_score)

15:23:51 - cmdstanpy - INFO - Chain [1] start processing
15:23:52 - cmdstanpy - INFO - Chain [1] done processing


RMSE: 1107.9731783500572
RMSPE: 0.19557501312049033
